In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


In [3]:
# Import the "all_tables_cleaned" dataset
mls_df = pd.read_csv('Clean_csv/all_tables_cleaned.csv', encoding= 'unicode_escape')
mls_df

FileNotFoundError: [Errno 2] No such file or directory: 'Clean_csv/all_tables_cleaned.csv'

In [3]:
# Check the "mls_df" datatypes.
mls_df.dtypes

Pos                   int64
Team                 object
Year                  int64
Games_Played          int64
Wins                  int64
Losses                int64
Draws                 int64
Goals_For             int64
Goals_Against         int64
Goal_Differential     int64
Points                int64
Conference           object
dtype: object

It looks like there are only 2 columns with **categorical** values i.e., **"Team"** and **"Conference"** columns, also having consistent data type as *object*.

We should **generate a list of categorical variable** names using Python's "df.dtypes" property. In that case, we can use our variable list to perform the **one-hot encoding** *once*, rather than for each individual variable.

In [4]:
# Generate our categorical variable list
mls_cat = mls_df.dtypes[mls_df.dtypes == "object"].index.tolist()
mls_cat

['Team', 'Conference']

Before going to encode these 2 columns using Scikit-learn's OneHotEncoder module, we need to make sure that none of the categorical variables have **more than 10 unique** values and require **bucketing**.

In [5]:
# Check the number of unique values in each column
mls_df[mls_cat].nunique()

Team          28
Conference     1
dtype: int64

The *Conference* column has only one unique value, that means this column has no variation in data across the dataset. So, we may **drop** this column from *mls_cat* list or use the *Team* column only for further processing.

The *Team* column has 28 unique team / club names which we need to keep as it is, because we want to predict outcome for each individual team. So, no needs for bucketing and we're ready to use **OneHotEncoder**.

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(mls_df[mls_cat]))
encode_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [8]:
# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(mls_cat)
encode_df.head()

,Team_Atlanta United FC,Team_Austin FC,Team_CF Montral,Team_Chicago Fire FC,Team_Chivas USA,Team_Colorado Rapids,Team_Columbus Crew,Team_D.C. United,Team_FC Cincinnati,Team_FC Dallas,...,Team_Orlando City SC,Team_Philadelphia Union,Team_Portland Timbers,Team_Real Salt Lake,Team_San Jose Earthquakes,Team_Seattle Sounders FC,Team_Sporting Kansas City,Team_Toronto FC,Team_Vancouver Whitecaps FC,Conference_Overall
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset using *panda's* **merge** and **drop** methods.

In [9]:
# Merge OneHotEncoded features and drop the originals from the updated df
mls_df = mls_df.merge(encode_df, left_index=True, right_index=True).drop(mls_df[mls_cat],1)
mls_df.head()

,Pos,Year,Games_Played,Wins,Losses,Draws,Goals_For,Goals_Against,Goal_Differential,Points,...,Team_Orlando City SC,Team_Philadelphia Union,Team_Portland Timbers,Team_Real Salt Lake,Team_San Jose Earthquakes,Team_Seattle Sounders FC,Team_Sporting Kansas City,Team_Toronto FC,Team_Vancouver Whitecaps FC,Conference_Overall
0,1,2004,30,12,5,13,40,32,8,49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,2004,30,14,9,7,38,30,8,49,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,3,2004,30,11,9,10,42,40,2,43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,2004,30,11,10,9,43,42,1,42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,2004,30,10,9,11,29,32,-3,41,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


The *mls_df* dataframe is **not showing all the columns**, as by default it shows 20 coumns. But we need to know all the Column headers to decide on testing/training data. To display all columns we can do the following,

In [10]:
# settings to display all columns
pd.set_option("display.max_columns", None)
mls_df.head(2)

,Pos,Year,Games_Played,Wins,Losses,Draws,Goals_For,Goals_Against,Goal_Differential,Points,Team_Atlanta United FC,Team_Austin FC,Team_CF Montral,Team_Chicago Fire FC,Team_Chivas USA,Team_Colorado Rapids,Team_Columbus Crew,Team_D.C. United,Team_FC Cincinnati,Team_FC Dallas,Team_Houston Dynamo FC,Team_Inter Miami CF,Team_LA Galaxy,Team_Los Angeles FC,Team_Minnesota United FC,Team_Nashville SC,Team_New England Revolution,Team_New York City FC,Team_New York City Red Bulls,Team_Orlando City SC,Team_Philadelphia Union,Team_Portland Timbers,Team_Real Salt Lake,Team_San Jose Earthquakes,Team_Seattle Sounders FC,Team_Sporting Kansas City,Team_Toronto FC,Team_Vancouver Whitecaps FC,Conference_Overall
0,1,2004,30,12,5,13,40,32,8,49,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,2004,30,14,9,7,38,30,8,49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


We need to **split** our **training** and **testing** data *before* fitting our **StandardScaler** instance. This <u> prevents testing data from influencing the standardization </u> function.

To build our training and testing datasets, we need to separate two values:

input values (which are our *independent variables* commonly referred to as **model features or "X"**) and **target output** ( *dependent variable* commonly referred to as **target or "y"** in TensorFlow documentation).

We want to build a model that will predict whether or not a team is winning; therefore, we must separate the "Wins" columns from the rest of the input data. Also, the "Conference_Overall" column has the same input for all the data, so we can drop this column.

In [12]:
# Split our preprocessed data into our features and target arrays
y = mls_df["Wins"].values
X = mls_df.drop(["Wins", "Conference_Overall"], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

Now that our training and testing data have been allocated, we're ready to build our **StandardScalerobject** and standardize the numerical features.

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Our data is now **preprocessed** via one-hot encoding and standardization. 

### Define Neural Network model

For our **input layer**, we must add the **number of input features equal to the number of variables in our feature** DataFrame.

In our **hidden layers**, we'll add **two hidden layers** with only a few neurons in each layer. To create the *second hidden layer*, we'll add another **Keras Dense class** while defining our model. All of our hidden layers will use the **relu activation** function to identify nonlinear characteristics from the input values.

In the **output layer**, we'll use the sigmoid activation function that will help us predict the probability that a team is winning or not.

In [14]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                380       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 441
Trainable params: 441
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
8/8 [==============================] - 1s 4ms/step - loss: -11.6011 - accuracy: 0.0041
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: -13.0681 - accuracy: 0.0041
Epoch 3/100
8/8 [==============================] - 0s 3ms/step - loss: -14.6220 - accuracy: 0.0041
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: -16.2216 - accuracy: 0.0041
Epoch 5/100
8/8 [==============================] - 0s 3ms/step - loss: -18.0280 - accuracy: 0.0041
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: -19.9743 - accuracy: 0.0041
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: -22.0362 - accuracy: 0.0041
Epoch 8/100
8/8 [==============================] - 0s 3ms/step - loss: -24.3531 - accuracy: 0.0041
Epoch 9/100
8/8 [==============================] - 0s 3ms/step - loss: -26.8798 - accuracy: 0.0041
Epoch 10/100
8/8 [==============================] - 0s 3ms/step - loss: -29.6523 - accuracy: 0.0041
Epoch 11/